In [1]:
# ================================================
# CREWAI TEST CASE GENERATOR (SAVES TO DOWNLOADS)
# ================================================
from dotenv import load_dotenv
load_dotenv()


from crewai import LLM, Agent, Crew, Task
import pandas as pd
import re
import os
from io import StringIO

# --- Create LLM instance ---
llm = LLM(model="gemini/gemini-2.0-flash", temperature=0.1)

# --- Agent 1: Requirement Analyzer ---
requirement_agent = Agent(
    name="RequirementAgent",
    role="Requirement Analyzer",
    goal="Analyze the acceptance criteria and extract all testable requirements.",
    backstory="You are an experienced QA Analyst skilled at identifying testable conditions and edge cases.",
    llm=llm
)

# --- Agent 2: Test Case Generator ---
testcase_agent = Agent(
    name="TestCaseAgent",
    role="Test Case Generator",
    goal="Generate structured test cases including name, precondition, description, expected result, and steps.",
    backstory="You are a QA automation expert who creates reusable and well-structured test cases based on requirements.",
    llm=llm
)

# --- Agent 3: Reviewer & Formatter ---
reviewer_agent = Agent(
    name="ReviewerAgent",
    role="Reviewer and Formatter",
    goal="Review the test cases for clarity, grammar, and correct markdown formatting.",
    backstory="You are a senior QA reviewer who ensures test documentation quality and consistency.",
    llm=llm
)

# --- Task 1: Extract Requirements ---
task1 = Task(
    description=(
        "Extract all possible testable requirements from the following acceptance criteria:\n\n"
        "{acceptance_criteria}\n\n"
        "Focus on functional and edge cases."
    ),
    agent=requirement_agent,
    expected_output="A list of clear, testable requirements."
)

# --- Task 2: Generate Test Cases ---
task2 = Task(
    description=(
        "Based on the extracted requirements below:\n\n"
        "{task1.output}\n\n"
        "Write test cases in this exact format:\n"
        "| Test Case Name | Pre-condition | Test Case Description (Expected Result) | Test Steps |\n"
    ),
    agent=testcase_agent,
    expected_output="Test cases written in a markdown table format."
)

# --- Task 3: Review and Format ---
task3 = Task(
    description=(
        "Review and clean up the generated test cases for clarity, formatting, and structure:\n\n"
        "{task2.output}"
    ),
    agent=reviewer_agent,
    expected_output="Final formatted markdown table of validated test cases."
)

# --- Create Crew ---
crew = Crew(
    agents=[requirement_agent, testcase_agent, reviewer_agent],
    tasks=[task1, task2, task3],
    process="sequential",
    verbose=True
)

# --- Input Acceptance Criteria ---
acceptance_criteria = """
When the user opens the Metro by T-Mobile app, they should be able to make a payment successfully.

1. The user should be able to access the "Make Payment" screen from the main dashboard.
2. The user should see available payment methods (Credit/Debit Card, Bank Account, or Gift Card).
3. The user should be able to select a saved payment method or add a new one.
4. The app should validate payment details before submission.
5. If payment is successful, a confirmation message and receipt should be displayed.
6. If payment fails, an appropriate error message should appear (e.g., invalid card, insufficient funds, network issue).
7. The user should be able to view their payment history after completing a transaction.
8. The payment amount should match the current due balance shown on the account.
9. The app should handle session timeouts gracefully during payment.
10. The user should receive an in-app notification and email confirmation after a successful payment.
"""

# --- Run the Crew ---
result = crew.kickoff(inputs={"acceptance_criteria": acceptance_criteria})

# --- Handle CrewAI version differences safely ---
final_output = (
        getattr(result, "final_output", None)
        or getattr(result, "output", None)
        or getattr(result, "raw", None)
        or str(result)
)

print("\n✅ Final Generated Test Cases (Markdown):\n")
print(final_output)

# --- Extract Markdown Table ---
table_match = re.search(r"(\| *Test Case Name.*)", final_output, re.DOTALL)

if table_match:
    table_text = table_match.group(1).strip()
    lines = [line.strip() for line in table_text.splitlines() if line.strip().startswith("|")]

    # --- Split into rows and columns ---
    split_rows = [[col.strip() for col in line.split("|") if col.strip()] for line in lines]
    split_rows = [row for row in split_rows if not all(re.match(r"-+", cell) for cell in row)]

    max_cols = max(len(r) for r in split_rows)
    for r in split_rows:
        while len(r) < max_cols:
            r.append("")

    header = split_rows[0]
    rows = split_rows[1:]
    df = pd.DataFrame(rows, columns=header)

    # --- Add auto numbering ---
    df.insert(0, "TC_ID", [f"TC{str(i + 1).zfill(3)}" for i in range(len(df))])

    # --- Save to Downloads folder ---
    downloads_dir = os.path.join(os.path.expanduser("~"), "Downloads")
    os.makedirs(downloads_dir, exist_ok=True)
    output_path = os.path.join(downloads_dir, "generated_test_cases.xlsx")

    df.to_excel(output_path, index=False)
    print(f"\n✅ Excel file saved successfully in your Downloads folder:\n{output_path}")

else:
    print("\n⚠️ No markdown table found in the output. Please check Crew output formatting.")


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9f309c22-5751-4507-bf5d-3015cc57bee6                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Requirement Analyzer                                                                                    │
│                                                                                                                 │
│  Task: Extract all possible testable requirements from the following acceptance criteria:                       │
│                                                                                                                 │
│                                                                                                                 │
│  When the user opens the Metro by T-Mobile app, they should be able to make a payment successfully.             │
│                                                                                                                 │
│  1. The user should be able to access the "Make Payment" screen from the main dashboard.                        │
│  2. The user should see available payment methods (Credit/Debit Card, Bank Account, or Gift Card).              │
│  3. The user should be able to select a saved payment method or add a new one.                                  │
│  4. The app should validate payment details before submission.                                                  │
│  5. If payment is successful, a confirmation message and receipt should be displayed.                           │
│  6. If payment fails, an appropriate error message should appear (e.g., invalid card, insufficient funds,       │
│  network issue).                                                                                                │
│  7. The user should be able to view their payment history after completing a transaction.                       │
│  8. The payment amount should match the current due balance shown on the account.                               │
│  9. The app should handle session timeouts gracefully during payment.                                           │
│  10. The user should receive an in-app notification and email confirmation after a successful payment.          │
│                                                                                                                 │
│                                                                                                                 │
│  Focus on functional and edge cases.                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/satishmokkala/Testcase-Creation/.venv/lib/python3.13/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Requirement Analyzer                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Testable Requirements:**                                                                                     │
│                                                                                                                 │
│  **1. Accessing the "Make Payment" Screen:**                                                                    │
│                                                                                                                 │
│  *   **1.1** Verify that the "Make Payment" screen is accessible from the main dashboard.                       │
│  *   **1.2** Verify the navigation flow to the "Make Payment" screen is intuitive and user-friendly.            │
│  *   **1.3** Verify that the "Make Payment" option is visible and enabled under different user                  │
│  roles/permissions (if applicable).                                                                             │
│                                                                                                                 │
│  **2. Available Payment Methods:**                                                                              │
│                                                                                                                 │
│  *   **2.1** Verify that all supported payment methods (Credit/Debit Card, Bank Account, Gift Card) are         │
│  displayed on the "Make Payment" screen.                                                                        │
│  *   **2.2** Verify that the payment method icons/labels are accurate and recognizable.                         │
│  *   **2.3** Verify that the payment methods are displayed in a logical order (e.g., most frequently used       │
│  first).                                                                                                        │
│  *   **2.4** Verify that unsupported payment methods are not displayed.                                         │
│                                                                                                                 │
│  **3. Payment Method Selection and Addition:**                                                                  │
│                                                                                                                 │
│  *   **3.1** Verify that the user can select a saved payment method.                                            │
│  *   **3.2** Verify that the saved payment method details (e.g., last four digits of card, bank name) are       │
│  displayed correctly.                                                                                           │
│  *   **3.3** Verify that the user can add a new payment method.                                                 │
│  *   **3.4** Verify that the "Add New Payment Method" flow is clear and easy to follow.                         │
│  *   **3.5** Verify that the app supports adding multiple payment methods of each type (e.g., multiple credit   │
│  cards).                                                                                                        │
│  *   **3.6** Verify that there is a mechanism to edit or delete saved payment methods.                          │
│  *   **3.7** Verify that newly added payment methods are saved securely and displayed correctly.                │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 09e0c512-534d-4991-a4a1-5c2cb247e52f                                                                     │
│  Agent: Requirement Analyzer                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...


✅ Final Generated Test Cases (Markdown):

```
| Test Case Name | Pre-condition | Test Case Description (Expected Result) | Test Steps |
|---|---|---|---|
| **1.1 Access Make Payment Screen from Dashboard** | User is logged in and on the main dashboard. | Verify that the "Make Payment" screen is accessible from the main dashboard. (The "Make Payment" screen should open successfully.) | 1. Navigate to the main dashboard. 2. Locate the "Make Payment" option. 3. Click on the "Make Payment" option. 4. Verify that the "Make Payment" screen is displayed. |
| **1.2 Navigation Flow to Make Payment Screen** | User is logged in and on the main dashboard. | Verify the navigation flow to the "Make Payment" screen is intuitive and user-friendly. (The user should be able to easily find and access the "Make Payment" screen.) | 1. Navigate to the main dashboard. 2. Attempt to locate the "Make Payment" option using different navigation paths (if available). 3. Assess the clarity and ease of use of each

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Reviewer and Formatter                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  | Test Case Name | Pre-condition | Test Case Description (Expected Result) | Test Steps |                      │
│  |---|---|---|---|                                                                                              │
│  | **1.1 Access Make Payment Screen from Dashboard** | User is logged in and on the main dashboard. | Verify    │
│  that the "Make Payment" screen is accessible from the main dashboard. (The "Make Payment" screen should open   │
│  successfully.) | 1. Navigate to the main dashboard. 2. Locate the "Make Payment" option. 3. Click on the       │
│  "Make Payment" option. 4. Verify that the "Make Payment" screen is displayed. |                                │
│  | **1.2 Navigation Flow to Make Payment Screen** | User is logged in and on the main dashboard. | Verify the   │
│  navigation flow to the "Make Payment" screen is intuitive and user-friendly. (The user should be able to       │
│  easily find and access the "Make Payment" screen.) | 1. Navigate to the main dashboard. 2. Attempt to locate   │
│  the "Make Payment" option using different navigation paths (if available). 3. Assess the clarity and ease of   │
│  use of each navigation path. 4. Verify that the most direct path is easily identifiable. |                     │
│  | **1.3 Make Payment Option Visibility (Admin User)** | User is logged in as an administrator. | Verify that   │
│  the "Make Payment" option is visible and enabled for an administrator. (The "Make Payment" option should be    │
│  visible and clickable.) | 1. Log in as an administrator. 2. Navigate to the main dashboard. 3. Locate the      │
│  "Make Payment" option. 4. Verify that the "Make Payment" option is visible and enabled. |                      │
│  | **1.3 Make Payment Option Visibility (Standard User)** | User is logged in as a standard user. | Verify      │
│  that the "Make Payment" option is visible and enabled for a standard user. (The "Make Payment" option should   │
│  be visible and clickable.) | 1. Log in as a standard user. 2. Navigate to the main dashboard. 3. Locate the    │
│  "Make Payment" option. 4. Verify that the "Make Payment" option is visible and enabled. |                      │
│  | **2.1 Supported Payment Methods Displayed** | User is on the "Make Payment" screen. | Verify that all        │
│  supported payment methods (Credit/Debit Card, Bank Account, Gift Card) are displayed. (All three payment       │
│  methods should be visible.) | 1. Navigate to the "Make Payment" screen. 2. Verify that "Credit/Debit Card",    │
│  "Bank Account", and "Gift Card" payment options are displayed. |                                               │
│  | **2.2 Payment Method Icons/Labels Accuracy** | User is on the "Make Payment" screen. | Verify that the       │
│  payment method icons/labels are accurate and recognizable. (The icons and labels should clearly represent      │
│  each payment method.) | 1. Navigate to the "Make Payment" screen. 2. Examine the icons and labels for each     │
│  payment method. 3. Verify that the icons are visually appropriate and the labels are accurate. |               │
│  | **2.3 Payment Methods Display Order** | User is on the "Make Payment" screen. | Verify that the payment      │
│  methods are displayed in a logical order (e.g., most f

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4aec201b-203b-4439-8fc5-2aca44b42d07                                                                     │
│  Agent: Reviewer and Formatter                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9f309c22-5751-4507-bf5d-3015cc57bee6                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```                                                                                              │
│  | Test Case Name | Pre-condition | Test Case Description (Expected Result) | Test Steps |                      │
│  |---|---|---|---|                                                                                              │
│  | **1.1 Access Make Payment Screen from Dashboard** | User is logged in and on the main dashboard. | Verify    │
│  that the "Make Payment" screen is accessible from the main dashboard. (The "Make Payment" screen should open   │
│  successfully.) | 1. Navigate to the main dashboard. 2. Locate the "Make Payment" option. 3. Click on the       │
│  "Make Payment" option. 4. Verify that the "Make Payment" screen is displayed. |                                │
│  | **1.2 Navigation Flow to Make Payment Screen** | User is logged in and on the main dashboard. | Verify the   │
│  navigation flow to the "Make Payment" screen is intuitive and user-friendly. (The user should be able to       │
│  easily find and access the "Make Payment" screen.) | 1. Navigate to the main dashboard. 2. Attempt to locate   │
│  the "Make Payment" option using different navigation paths (if available). 3. Assess the clarity and ease of   │
│  use of each navigation path. 4. Verify that the most direct path is easily identifiable. |                     │
│  | **1.3 Make Payment Option Visibility (Admin User)** | User is logged in as an administrator. | Verify that   │
│  the "Make Payment" option is visible and enabled for an administrator. (The "Make Payment" option should be    │
│  visible and clickable.) | 1. Log in as an administrator. 2. Navigate to the main dashboard. 3. Locate the      │
│  "Make Payment" option. 4. Verify that the "Make Payment" option is visible and enabled. |                      │
│  | **1.3 Make Payment Option Visibility (Standard User)** | User is logged in as a standard user. | Verify      │
│  that the "Make Payment" option is visible and enabled for a standard user. (The "Make Payment" option should   │
│  be visible and clickable.) | 1. Log in as a standard user. 2. Navigate to the main dashboard. 3. Locate the    │
│  "Make Payment" option. 4. Verify that the "Make Payment" option is visible and enabled. |                      │
│  | **2.1 Supported Payment Methods Displayed** | User is on the "Make Payment" screen. | Verify that all        │
│  supported payment methods (Credit/Debit Card, Bank Account, Gift Card) are displayed. (All three payment       │
│  methods should be visible.) | 1. Navigate to the "Make Payment" screen. 2. Verify that "Credit/Debit Card",    │
│  "Bank Account", and "Gift Card" payment options are displayed. |                                               │
│  | **2.2 Payment Method Icons/Labels Accuracy** | User is on the "Make Payment" screen. | Verify that the       │
│  payment method icons/labels are accurate and recognizable. (The icons and labels should clearly represent      │
│  each payment method.) | 1. Navigate to the "Make Payment" screen. 2. Examine the icons and labels for each     │
│  payment method. 3. Verify that the icons are visually appropriate and the labels are accurate. |               │
│  | **2.3 Payment Methods Display Order** | User is on 


✅ Excel file saved successfully in your Downloads folder:
/Users/satishmokkala/Downloads/generated_test_cases.xlsx
